In [1]:
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pydotplus
import graphviz
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('parkinsons_updrs.csv',header = None)
data.columns = {'subject':int,
               'age':int,
               'sex':int,
               'test_time':int,
                'motor_UPDRS':int,
                'total_UPDRS':int,
               'Jitter_p':int,
               'Jitter_Abs':int,
               'JitterRAP':int,
               'JitterPPQ5':int,
               'JitterDDP':int,
               'Shimmer':int,
               'ShimmerAPQ3':int,
               'ShimmerAPQ5':int,
               'ShimmerAPQ11':int,
               'ShimmerDDA':int,
               'NHR':int,
                'HNR':int,
                'RPDE':int,
                'DFA':int,
                'PPE':int
                
               }
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'parkinsons_updrs.csv'

In [3]:
le=LabelEncoder()
for col in data[['subject',
               'age',
               'sex',
               'test_time',
                'motor_UPDRS',
                'total_UPDRS',
               'JitterRAP',
               'JitterPPQ5',
               'JitterDDP',
               'Shimmer',
               'ShimmerAPQ3',
               'ShimmerAPQ5',
               'ShimmerAPQ11',
                 'Jitter_p',
                 'Jitter_Abs',
               'ShimmerDDA',
               'NHR',
                'HNR',
                'RPDE',
                'DFA',
                'PPE']]:
    data[col]=le.fit_transform(data[col])
data.info()

NameError: name 'data' is not defined

In [4]:
data.replace(" ?", np.nan, inplace= True)
new_data = data.dropna()
print(r'總比數紀錄:{}'.format(data.shape[0]))
print(r'刪除含缺漏紀錄資料後比數:{}'.format(data.dropna().shape[0]))

pd.get_dummies(new_data['subject'])
subject_encoding = pd.get_dummies(new_data['subject'], prefix= 'subject')
new_data = new_data.drop('subject', 1)

pd.get_dummies(new_data['Jitter_Abs'])
Jitter_Abs_encoding = pd.get_dummies(new_data['Jitter_Abs'], prefix= 'Jitter_Abs')

pd.get_dummies(new_data['Shimmer'])
Shimmer_encoding = pd.get_dummies(new_data['Shimmer'], prefix= 'Shimmer')

pd.get_dummies(new_data['DFA'])
DFA_encoding = pd.get_dummies(new_data['DFA'], prefix= 'DFA')

pd.get_dummies(new_data['PPE'])
PPE_encoding = pd.get_dummies(new_data['PPE'], prefix= 'PPE')

pd.get_dummies(new_data['age'])
age_encoding = pd.get_dummies(new_data['age'], prefix= 'age')

pd.get_dummies(new_data['HNR'])
HNR_encoding = pd.get_dummies(new_data['HNR'], prefix= 'HNR')

pd.get_dummies(new_data['motor_UPDRS'])
motor_UPDRS_encoding = pd.get_dummies(new_data['motor_UPDRS'], prefix= 'motor_UPDRS')

pd.get_dummies(new_data['test_time'])
test_time_encoding = pd.get_dummies(new_data['test_time'], prefix= 'test_time')

pd.get_dummies(new_data['ShimmerAPQ3'])
ShimmerAPQ3_encoding = pd.get_dummies(new_data['ShimmerAPQ3'], prefix= 'ShimmerAPQ3')

pd.get_dummies(new_data['ShimmerAPQ11'])
ShimmerAPQ11_encoding = pd.get_dummies(new_data['ShimmerAPQ11'], prefix= 'ShimmerAPQ11')

pd.get_dummies(new_data['JitterDDP'])
JitterDDP_encoding = pd.get_dummies(new_data['JitterDDP'], prefix= 'JitterDDP')

pd.get_dummies(new_data['JitterRAP'])
JitterRAP_encoding = pd.get_dummies(new_data['JitterRAP'], prefix= 'JitterRAP')

pd.get_dummies(new_data['JitterPPQ5'])
JitterPPQ5_encoding = pd.get_dummies(new_data['JitterPPQ5'], prefix= 'JitterPPQ5')

pd.get_dummies(new_data['RPDE'])
RPDE_encoding = pd.get_dummies(new_data['RPDE'], prefix= 'RPDE')

pd.get_dummies(new_data['ShimmerAPQ5'])
ShimmerAPQ5_encoding = pd.get_dummies(new_data['ShimmerAPQ5'], prefix= 'ShimmerAPQ5')

pd.get_dummies(new_data['Jitter_p'])
Jitter_p_encoding = pd.get_dummies(new_data['Jitter_p'], prefix= 'Jitter_p')

pd.get_dummies(new_data['sex'])
sex_encoding = pd.get_dummies(new_data['sex'], prefix= 'sex')


pd.get_dummies(new_data['NHR'])
NHR_encoding = pd.get_dummies(new_data['NHR'], prefix= 'NHR')

pd.get_dummies(new_data['ShimmerDDA'])
ShimmerDDA_encoding = pd.get_dummies(new_data['ShimmerDDA'], prefix= 'ShimmerDDA')

pd.get_dummies(new_data['total_UPDRS'])
total_UPDRS_encoding = pd.get_dummies(new_data['total_UPDRS'], prefix= 'total_UPDRS')


pd.DataFrame(new_data)

NameError: name 'data' is not defined

In [5]:
data2 = preprocessing.normalize(new_data, norm= 'l2')
scaler = MinMaxScaler()
data3 = scaler.fit(new_data)
data3 = scaler.transform(new_data)

NameError: name 'new_data' is not defined

In [6]:
x= new_data.drop(['total_UPDRS'], axis = 1)
x_norm = preprocessing.normalize(x, norm= 'l2')
y = new_data['total_UPDRS']

NameError: name 'new_data' is not defined

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.20)
x_train.shape

NameError: name 'x' is not defined

In [8]:
dtree = DecisionTreeClassifier(criterion="entropy")
dtree.fit(x_train, y_train)
#預測 y_test
y_test_predicted = dtree.predict(x_test)
#結果
accuracy = metrics.accuracy_score(y_test, y_test_predicted)
print(accuracy)

NameError: name 'x_train' is not defined

In [9]:
score_array= []
score_array_test= []

for i in range(1,30):
    dtree = DecisionTreeClassifier(criterion="entropy", max_depth= i)
    dtree.fit(x_train, y_train)
    score_val = dtree.score(x_train, y_train)
    score_test = dtree.score(x_test, y_test)
    score_array.append(score_val)
    score_array_test.append(score_test)

print("score_array",score_array)
print(" ")
print("score_array_MAX:", np.max(score_array))
print("----------------------------------------------------------------------------------------------------------")
print("score_array_test", score_array_test)
print(" ")
print("score_array_test_MAX:", np.max(score_array_test))
plt.plot(score_array)
plt.plot(score_array_test)

NameError: name 'x_train' is not defined

In [10]:
# 建立樹的圖表
dot_tree = tree.export_graphviz(dtree,out_file=None)
graphviz = pydotplus.graph_from_dot_data(dot_tree)
# 輸出預測結果的圖檔
graphviz.write_pdf("parkinsons_tree.pdf")

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.